# Домашнее задание
## Домашнее задание №6 - Продвинутая оптимизация
### Задание

Проведите следующий эксперимент: для функции Растригина найдите среднее и минимальное значение целевой функции по результатам 100 запусков процесса минимизации, а также среднее число итераций при случайных начальных условиях в диапазоне [-5, 5].

Эксперимент необходимо провести для размерностей 2, 3, 4.

f(x) = A*n + SUM(Xi**2 - A*cos(2*Pi*Xi))

Целью данного вычислительного эксперимента является выбор наиболее эффективного метода для данной задачи. Исследование нужно выполнить с использованием метода Дифференциальной эволюции (значения параметра recombination 0.3 и 0.7), а также встроенных методов функции Minimize:

- Nelder-Mead
- BFGS
- Newton-CG



In [1]:
# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import sympy as sym
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

In [5]:
# Для самопроверки определяем производную функции Растригина по одной из переменных (в случае, если А=10)
# Производные потребуются далее для формирования функции, возвращающей вектор градиента для метода оптимизации Newton-CG
XX = [sym.Symbol('x1'), sym.Symbol('x2'), sym.Symbol('x3')]
# Определяем функцию
fnc = 20 + XX[0]**2 - 10*sym.cos(2*np.pi*XX[0]) + XX[1]**2 - 10*sym.cos(2*np.pi*XX[1]) + XX[2]**2 - 10*sym.cos(2*np.pi*XX[2])
print(fnc)
res=sym.diff(fnc, XX[0])
print(res)

x1**2 + x2**2 + x3**2 - 10*cos(6.28318530717959*x1) - 10*cos(6.28318530717959*x2) - 10*cos(6.28318530717959*x3) + 20
2*x1 + 62.8318530717959*sin(6.28318530717959*x1)


In [7]:
# Вначале определяем целевую функцию
# Поскольку везде пишут, что A=10, по умолчанию зададим это значение.
# Формулу перепишем, затащив A под сумму, чтобы минимизировать код функции
def rastrigin(X, A=10):
    return sum(list(map(lambda x: x**2 + A*(1 - np.cos(2*np.pi*x)), X)))

# Функция для вычисления вектора градиента функции Растригина для заданной точки X
def jacobian(X):
    jac = []
    for xx in X:
         jac.append(20*np.pi*np.sin(2*np.pi*xx) + 2*xx)
    return jac


# Определяем для размерностей 2, 3 и 4 соответственно диапазоны значений
bound = (-5, 5)
boundaries = [[], [], [bound, bound], [bound, bound, bound], [bound, bound, bound, bound]]
jacobian([0,0,0])

[0.0, 0.0, 0.0]

In [9]:
# Определяем функции для вычисления минимума для функции rastrigin для каждого из пяти методов, которые мы сравниваем
# Метод Нелдера-Меда
def min_nelder_mead(X):
    return minimize(rastrigin, X, method='Nelder-Mead', options={'xtol': 1e-1, 'disp': False}, bounds=boundaries[len(X)])

# Метод BFGS
def min_bfgs(X):
    return minimize(rastrigin, X, method='BFGS', jac=jacobian, options={'gtol': 1e-1, 'disp': False}, bounds=boundaries[len(X)])

# Метод Newton-CG
def min_newton_cg(X):
    return minimize(rastrigin, X, method='Newton-CG', jac=jacobian, options={'xtol': 1e-1, 'disp': False})

# Метод Дифференциальной эволюции, recombinatio=0.3
def min_dif_evol_03(X):
    return differential_evolution(rastrigin, boundaries[len(X)], seed=21,tol=0.1,recombination=0.3)

# Метод Дифференциальной эволюции, recombinatio=0.7
def min_dif_evol_07(X):
    return differential_evolution(rastrigin, boundaries[len(X)], seed=21,tol=0.1,recombination=0.7)

min_functions = {
    'Nelder-Mead': min_nelder_mead, 
    'BFGS': min_bfgs, 
    'Newton_CG': min_newton_cg, 
    'Differencial Evolution 03': min_dif_evol_03, 
    'Differencial Evolution 07': min_dif_evol_07
}


In [11]:
# Попытался основное тело программы сделать максимально компактным.
for method_name, min_func in min_functions.items():
    print(f"\n\n{method_name}")
    for dimension in [2,3,4]:
        itList=[]
        print(f"\nДля размерности {dimension}:")
        for kk in range(100):
            X=np.random.uniform(low=-5,high=5,size=(dimension,))
            res = min_func(X)

            if 'Dif' in method_name:
                itList.append(res.fun)    
            else:
                itList.append(res.nit)
            
        print(f'Среднее значение {sum(itList)/len(itList)}, Минимальное значение {min(itList)}')



Nelder-Mead

Для размерности 2:
Среднее значение 24.4, Минимальное значение 15

Для размерности 3:
Среднее значение 47.79, Минимальное значение 26

Для размерности 4:
Среднее значение 85.84, Минимальное значение 44


BFGS

Для размерности 2:
Среднее значение 5.41, Минимальное значение 3

Для размерности 3:
Среднее значение 7.18, Минимальное значение 5

Для размерности 4:
Среднее значение 10.14, Минимальное значение 7


Newton_CG

Для размерности 2:
Среднее значение 2.47, Минимальное значение 0

Для размерности 3:
Среднее значение 2.36, Минимальное значение 0

Для размерности 4:
Среднее значение 2.48, Минимальное значение 1


Differencial Evolution 03

Для размерности 2:
Среднее значение 0.0, Минимальное значение 0.0

Для размерности 3:
Среднее значение 0.0, Минимальное значение 0.0

Для размерности 4:
Среднее значение 0.0, Минимальное значение 0.0


Differencial Evolution 07

Для размерности 2:
Среднее значение 0.0, Минимальное значение 0.0

Для размерности 3:
Среднее значение 0.0, М

In [13]:
# Мне очень не нравятся полученные результаты, но может быть, они и не могли получиться другими?
# Метод Нелдера-Мида вообще неадекватен, а полученные с его помощью цифры кажутся мне по-меньшей мере странными
# Возможно, я выбрал неудачные параметры или просто не указал что-то важное при вызове функции.
# 
# Дифференциальная эволюция одинаково эффективно сработала для всех вариантов размерности и для всех вариантов параметра
# recombination, выдав правильные результаты. Но для каждой размерности сто запусков функции занимали минуты полторы.
# Это тоже кажется мне ненормальным и, возможно, связано также с неудачно подобранными параметрами.
#
# Буду благодарен за любые комментарии и рекомендации в этой связи.